# Model Analyses and Interpretation

So far, I have trained two machined learning models to explain the impact of access to transit on the proportion of residents using public transportation, down to the DA level. I also tuned their hyperparameters using cross validation. They have both achieved cross-validation $R^2$ at about 0.67. 

However, there is still a big distance between our models and actionable real-world solutions. At the end of the day, we want to know where the transit authority of Greater Vancouver Area should put resource to develop new infrastructure. 

In [14]:
import geopandas
import json
import joblib
import os
import pandas as pd

In [26]:
# sklearn modules
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    MinMaxScaler,
)

In [77]:
# Mapping modules
import contextily as ctx
import matplotlib.pyplot as plt

GVA_map_xlim_lower = -13746072.435927173
GVA_map_xlim_higher = -13630000
GVA_map_ylim_lower = 6270302.809935683
GVA_map_ylim_higher = 6345000

In [16]:
# Re-define working directory
data_version = "20200606"
cwd = os.path.dirname(os.getcwd())
cwd = os.path.dirname(os.getcwd())
os.chdir(cwd)
data_dir = os.path.join(os.getcwd(), "TL_data", data_version)

In [23]:
## Data
X_train = geopandas.read_file(
    os.path.join(os.getcwd(), "Data_Tables", data_version, "X_train.json")
)

In [38]:
with open(os.path.join(os.getcwd(), "Data_Tables", data_version, "X_header.json"), "r") as X_header_outfile:
    X_header = json.load(X_header_outfile)
    
with open(os.path.join(os.getcwd(), "Data_Tables", data_version, "y_train.json"), "r") as y_train_outfile:
    y_train = json.load(y_train_outfile)
y_train = pd.read_json(y_train, typ="series")

In [18]:
# Read data and models

## Trained models
random_search_rf = joblib.load(os.path.join(
        os.getcwd(),
        "Models",
        data_version,
        "random_search_rf.joblib",
    ))
random_search_LASSO = joblib.load(os.path.join(
        os.getcwd(),
        "Models",
        data_version,
        "random_search_LASSO.joblib",
    ))

In [52]:
## Pre-processor 
categorical_transformer, numeric_transformer, proportion_transformer, ColumnTransformer = joblib.load(
    os.path.join(os.getcwd(), "Models", data_version, "preprocessor.joblib")
)


## Variable types
with open(
    os.path.join(os.getcwd(), "Models", data_version, "features.json"),
    "r",
) as feature_outfile:
    categorical_features, numeric_features, proportion_features, geometry_feature = json.load(feature_outfile)

## LASSO model
pipe_LASSO = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("LASSO_reg", Lasso()),
    ]
)
## Random Forest model
pipe_rf = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("rf_reg", RandomForestRegressor()),
    ]
)

In [59]:
## Get input feature names
def get_column_names_from_ColumnTransformer(column_transformer):

    col_name = []

    for transformer_in_columns in column_transformer.transformers_[
        :-1
    ]:  # the last transformer is ColumnTransformer's 'remainder'
        print("\n\ntransformer: ", transformer_in_columns[0])

        raw_col_name = list(transformer_in_columns[2])

        if isinstance(transformer_in_columns[1], Pipeline):
            # if pipeline, get the last transformer
            transformer = transformer_in_columns[1].steps[-1][1]
        else:
            transformer = transformer_in_columns[1]

        try:
            if isinstance(transformer, OneHotEncoder):
                names = list(transformer.get_feature_names(raw_col_name))

            elif isinstance(transformer, SimpleImputer) and transformer.add_indicator:
                missing_indicator_indices = transformer.indicator_.features_
                missing_indicators = [
                    raw_col_name[idx] + "_missing_flag"
                    for idx in missing_indicator_indices
                ]

                names = raw_col_name + missing_indicators

            else:
                names = list(transformer.get_feature_names())

        except AttributeError as error:
            names = raw_col_name

        col_name.extend(names)

    return col_name


preprocessor.fit(X_train)
feature_names = get_column_names_from_ColumnTransformer(preprocessor)

categorical_feature_names = list(categorical_transformer["onehot"].get_feature_names(categorical_features))
numeric_feature_names = numeric_features
proportion_feature_names = proportion_features



transformer:  cat


transformer:  num


transformer:  prop


## Feature Importance
Both LASSO and Random Forest models give easy access to measurements of global feature importances. 
For LASSO model, I use the magnificance of coefficients to roughly estimate each variable's importance. 
For Random Forest model, I use the [impurity measurement](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.feature_importances_) of each variable's importance. 

In addition, the three types of input variables, namely `categorical_features`, `numeric_features`, and `proportion_features` are scaled differently in the `preprocessing` step. Therefore, I will review the important variables for all three categories separately. 

In [32]:
### LASSO Model
LASSO_coeffs = random_search_LASSO.best_estimator_["LASSO_reg"].coef_

LASSO_feature_coeffs = pd.DataFrame({"feature": feature_names, "coeffs": LASSO_coeffs})

### LASSO Model
The table below shows the five categorical features that mostly strongly predict proportion of transit use among residents. They are ADA or CCS areas. 

In [67]:
#### Categorical features
LASSO_categorical_feature_coeffs = LASSO_feature_coeffs.loc[LASSO_feature_coeffs["feature"].isin(categorical_feature_names), :]
LASSO_categorical_feature_coeffs.sort_values("coeffs", ascending = False).head(5)

,feature,coeffs
148,ADAUID_59150117,0.097663
7,CCSUID_5915022,0.069961
8,CCSUID_5915025,0.065468
138,ADAUID_59150107,0.053888
114,ADAUID_59150082,0.048754


In [ ]:
X_train_LASSO_top_categorical_mask = ((X_train.ADAUID == "59150117") 
                                      | (X_train.CCSUID == "5915022") 
                                      | (X_train.CCSUID == "5915025") 
                                      | (X_train.ADAUID == "59150107") 
                                      | (X_train.ADAUID == "59150082"))

X_train_LASSO_top_categorical = X_train.loc[X_train_LASSO_top_categorical_mask, :]

LASSO_top_categorical_ax = X_train_LASSO_top_categorical.plot(
    figsize=(20, 20),
    alpha=0.5,
)
LASSO_top_categorical_ax.set_xlim(GVA_map_xlim_lower, GVA_map_xlim_higher)
LASSO_top_categorical_ax.set_ylim(GVA_map_ylim_lower, GVA_map_ylim_higher)

ctx.add_basemap(LASSO_top_categorical_ax, zoom=12)
plt.title("Removed DAs", fontsize=30)

In [ ]:
plt.savefig(
    os.path.join(
        os.getcwd(),
        "Vancouver_transit",
        "Maps",
        data_version,
        "DA_removed.png",
    )
)

In [74]:
(X_train.ADAUID == "59150117") | (X_train.CCSUID == "5915022") | (X_train.CCSUID == "5915025") | (X_train.ADAUID == "59150107") | (X_train.ADAUID == "59150082") 

0       False
1       False
2        True
3       False
4       False
        ...  
2539    False
2540     True
2541     True
2542     True
2543    False
Length: 2544, dtype: bool

In [68]:
LASSO_categorical_feature_coeffs.sort_values("coeffs").head(5)

,feature,coeffs
70,ADAUID_59150035,-0.119335
74,ADAUID_59150040,-0.092524
76,ADAUID_59150042,-0.071774
69,ADAUID_59150034,-0.060664
313,ADAUID_59150312,-0.056735


In [49]:
LASSO_feature_coeffs["feature"].str.startswith("C")

0       False
1        True
2        True
3        True
4        True
        ...  
1367    False
1368    False
1369    False
1370    False
1371    False
Name: feature, Length: 1372, dtype: bool

In [40]:
categorical_features

['PRUID', 'CDUID', 'CCSUID', 'CSDUID', 'ERUID', 'CMAUID', 'CMAPUID', 'ADAUID']

In [30]:
# Analyze random forest model

## Variable importance
rf_immpurity_feat_imp = random_search_rf.best_estimator_["rf_reg"].feature_importances_

rf_immpurity_feat_imp_coeffs = pd.DataFrame(
    {"feature": feature_names, "impurity_importance": rf_immpurity_feat_imp}
)

In [31]:
rf_immpurity_feat_imp_coeffs

,feature,impurity_importance
0,PRUID_59,0.000000
1,CDUID_5915,0.000000
2,CCSUID_5915001,0.004445
3,CCSUID_5915004,0.000198
4,CCSUID_5915011,0.000015
...,...,...
1367,vn473_immediate_vn471,0.000950
1368,vn474_immediate_vn473,0.000481
1369,vn475_immediate_vn474,0.000727
1370,vn476_immediate_vn474,0.000560


In [ ]:
## How I built the two models

As it is markdown, you can embed images, HTML, etc into your posts!

![](https://myst-parser.readthedocs.io/en/latest/_static/logo.png)

You an also $add_{math}$ and

$$
math^{blocks}
$$

or

$$
\begin{aligned}
\mbox{mean} la_{tex} \\ \\
math blocks
\end{aligned}
$$

But make sure you \$Escape \$your \$dollar signs \$you want to keep!

## MyST markdown

MyST markdown works in Jupyter Notebooks as well. For more information about MyST markdown, check
out [the MyST guide in Jupyter Book](https://jupyterbook.org/content/myst.html),
or see [the MyST markdown documentation](https://myst-parser.readthedocs.io/en/latest/).

## Code blocks and outputs

Jupyter Book will also embed your code blocks and output in your book.
For example, here's some sample Matplotlib code:

In [1]:
from matplotlib import rcParams, cycler
import matplotlib.pyplot as plt
import numpy as np

plt.ion()

In [ ]:
# Fixing random state for reproducibility
np.random.seed(19680801)

N = 10
data = [np.logspace(0, 1, 100) + np.random.randn(100) + ii for ii in range(N)]
data = np.array(data).T
cmap = plt.cm.coolwarm
rcParams['axes.prop_cycle'] = cycler(color=cmap(np.linspace(0, 1, N)))


from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color=cmap(0.), lw=4),
                Line2D([0], [0], color=cmap(.5), lw=4),
                Line2D([0], [0], color=cmap(1.), lw=4)]

fig, ax = plt.subplots(figsize=(10, 5))
lines = ax.plot(data)
ax.legend(custom_lines, ['Cold', 'Medium', 'Hot']);

There is a lot more that you can do with outputs (such as including interactive outputs)
with your book. For more information about this, see [the Jupyter Book documentation](https://jupyterbook.org)